Gaussian Process Regression
===========================

In [1]:
%matplotlib inline

import plotting
import utils

import matplotlib.pyplot as plt
import numpy as np
from sklearn.gaussian_process import GaussianProcessRegressor as GPR
from sklearn.gaussian_process import kernels

**1D case**

In [ ]:
# Define function
def f_1D(x):
    return np.sin(2*np.pi*x) * np.exp(3*x)

In [ ]:
# Setup data points
xs = np.linspace(0, 1, num=7)
fs = f_1D(xs)
ys = fs

In [ ]:
# Setup GaussianProcessRegressor
kernel = kernels.RBF(length_scale=0.05)
gpr = GPR(kernel=kernel,
          alpha=1e-4,
          optimizer=None,
          n_restarts_optimizer=30)
#gpr.fit(xs[:, np.newaxis], ys)

In [ ]:
# Plot
plotting.plot_gpr_1d(gpr, with_kernel=True, with_lml=False, n_samples=10)
plt.tight_layout()

**Exercises:**
1. Fit the function to the data points.
(Comment in `gpr.fit(...)`.)
Plot again.
2. The current length scale hyperparameter is not suitable for the give data points.
Optimize the hyperparameter (i.e., the length scale).
(Comment out `optimizer=None`.)
Plot again.
How did the length scale change?
3. Change the number of data points.
What is the minimum number of data points that leads to a "reasonable" approximation?
4. Add noise to the data `ys` (Gaussian, zero mean, fixed standard deviation).
Let the GPR know about it by adjusting the `alpha` parameter in the constructor to the noise standard deviation.
Fit and plot again.
What has changed with the approximation?
5. Try different kernels from the `kernels` module.
See [https://scikit-learn.org/stable/modules/gaussian_process.html#kernels-for-gaussian-processes](https://scikit-learn.org/stable/modules/gaussian_process.html#kernels-for-gaussian-processes).

**Solutions:**

2. The length scale changes to ~0.109.
3. The minimum number is 6.
With <=5 data points, we get unreasonable approximations.
4. `ys = fs + rnd.normal(scale=0.3, size=len(xs))`, `alpha=0.3`.
The approximation (the posterior mean function) is not interpolating the data points any more due to their noise (uncertainty).
5. `kernel = kernels.Matern(length_scale=0.05, nu=1.5)`.
(`nu` is the "smoothness" parameter. `nu=1.5` means that the function is assumed to be continuously differentiable once.)

**2D case**

In [ ]:
# Define function
def f_2D(x):
    x1 = x[..., 0]
    x2 = x[..., 1]

    return np.sin(2*np.pi*x1) * np.sin(2*np.pi*x2)

In [ ]:
# Setup data points
x1s = np.linspace(0, 1, num=10)
x2s = x1s
xs = utils.cartesian_product(x1s, x2s)
fs = f_2D(xs)
ys = fs

In [ ]:
# Setup GaussianProcessRegressor
kernel = kernels.RBF(length_scale=[1., 1.])
gpr = GPR(kernel=kernel,
          alpha=1e-2,
          n_restarts_optimizer=100)
gpr.fit(xs, ys)

In [ ]:
# Plot
plotting.contour_gpr_2d(gpr)
plt.tight_layout()

plotting.surf_gpr_2d(gpr)
plt.tight_layout()

**Exercises:**
1. What are the optimized length scales?
Are they similar/equal?
Why?
2. Increase the frequency of the function only along one direction.
Fit and plot again.
Are the optimized length scales still similar?
Do they have a relation related to the frequencies?
3. Decrease the number of data points.
What is the minimum number of data points that leads to a "reasonable" approximation?

**Solutions:**
1. Both length scales are 0.284.
The reason is that the function equally looks in both direction.
2. Doubling the frequency along the first axis with `np.sin(2*2*np.pi*x1) * ...` results in length scales 0.142 and 0.28, which reflects the first axis being twice as frequent as the second one.
3. With the original function (same frequency on both axes), a 4x4 grid already leads to unreasonable approximations.
Even a 5x5 grid might not be enough.
So, the minimum square grid is 6x6.